In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
import re

In [4]:
l = ['a', 'b', 'c']
l_ = ['d', 'e']
l+l_

['a', 'b', 'c', 'd', 'e']

In [19]:
test = 'Posted — March 25, 2022'
pattern = re.compile(r'\w+ — (\w+ \d+, \d{4})')
result = re.match(pattern, test)
result[1]

'March 25, 2022'

In [24]:
link = "https://urbandesignforum.org/review/our-stories/"
driver = webdriver.Firefox()
driver.get(link)
time.sleep(5)
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
corpus = pd.DataFrame(columns = ['title', 'url', 'date', 'text'])
pattern = re.compile(r'\w+ — (\w+ \d+, \d{4})')
result = re.match(pattern, test)
links = []
titles = []
dates = []
while True:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    headers = soup.select('div.title')
    date_headers = soup.select('h6.publish-date')
    links.extend([header.a['href'] for header in headers])
    titles.extend([header.a.h4.text for header in headers])
    dates.extend([re.match(pattern, header.text)[1] for header in date_headers])
    driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
    try:
        button = driver.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
        button.click()  
        time.sleep(1)
    except NoSuchElementException:
        break

# soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [33]:
corpus = pd.DataFrame()
corpus['title'] = titles
corpus['url'] = links
corpus['date'] = dates

In [35]:
r = requests.get(corpus.loc[0, 'url'])

In [37]:
soup = BeautifulSoup(r.text, 'html.parser')

In [46]:
text = [p.text.strip() for p in soup.select('div.entry-content.mb-5')]

In [47]:
text

['For this month’s Fellow Spotlight, we spoke with Jhordan Channer and Dorraine Duncan, co-founders of Island City Lab.\nIsland City Lab is a think-tank that confronts the defining issues affecting the built environment of island nations co-founded by Urban Design Forum Fellows Jhordan Channer and Dorraine Duncan. Aiming to develop a network of urbanists, Island City Lab sees the need for the creation and dissemination of best practices in the urban development of Caribbean towns and cities.\nA major goal for the organization will be to convene stakeholders and break down silos across design, real estate, transit, housing, sustainability, and policy sectors to share new ideas around major urban issues and interrogate commonly held ones.\nBelow, we talk with Jhordan and Dorraine about what’s to come from Island City Lab.\n\nAt Island City Lab, you’re convening stakeholders and sharing knowledge around critical issues affecting island cities. Why is there a need–specifically in the Carib

In [49]:
for i, link in tqdm(corpus.iterrows(), total=len(links)):
    try:
        url = link.url
        title = link.title
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        article = text = ' '.join([p.text.strip() for p in soup.select('div.entry-content.mb-5')])
        text = re.sub(r'[\r\n\xa0 ]+', ' ', article).strip()
        corpus.loc[i, 'text'] = text
    except Exception as e:
        corpus.loc[i] = np.nan
        print(f'Error at index {i}: {e}')

  0%|          | 0/81 [00:00<?, ?it/s]

In [50]:
corpus

,title,url,date,text
0,Fellows Spotlight: Dorraine Duncan and Jhordan...,https://urbandesignforum.org/fellow-spotlight-...,"March 25, 2022","For this month’s Fellow Spotlight, we spoke wi..."
1,Neighborhoods Now: In Conversation with Ryan G...,https://urbandesignforum.org/neighborhoods-now...,"April 30, 2021",We are excited to continue our Neighborhoods N...
2,Neighborhoods Now: In Conversation with Ian Gr...,https://urbandesignforum.org/neighborhoods-now...,"April 1, 2021",We are excited to continue our Neighborhoods N...
3,Neighborhoods Now: In Conversation with Yin Ko...,https://urbandesignforum.org/neighborhoods-now...,"March 24, 2021",We are excited to continue our Neighborhoods N...
4,Turning the Heat Collection of Proposals: Intr...,https://urbandesignforum.org/turning-the-heat-...,"December 15, 2020",The Urban Design Forum’s 2019 Forefront Fellow...
...,...,...,...,...
76,"Grid, Interrupted",https://urbandesignforum.org/grid-interrupted/,"June 14, 2012",Editors Note: As the Museum of the City of New...
77,Rewriting a City in Nature,https://urbandesignforum.org/rewriting-a-city-...,"February 15, 2012",Editor’s Note: Check out our review of Diana B...
78,No School is an Island,https://urbandesignforum.org/no-school-is-an-i...,"February 9, 2012",Edtior’s Note: The vision of the triumphant Co...
79,Rurbanism: A New Model for African Development,https://urbandesignforum.org/7521-2/,"December 19, 2011","It is projected that by 2025, 200 million Afri..."


In [51]:
corpus.to_csv('./urbandesignforum_review_our-stories_corpus.csv')

In [54]:
corpus.loc[2, 'text']

'We are excited to continue our Neighborhoods Now initiative with Asian Americans for Equality (AAFE) and Think!Chinatown in Chinatown and Banana Kelly in the South Bronx. Additionally, we expanded our partnership with Fourth Arts Block in the Lower East Side. These working groups will reenergize outdoor spaces to support community programming and cultural revitalization. These next few weeks, we will be rolling out interviews with the representatives of each organization to learn more about their history, some neighborhood insights, and what they hope to achieve through Neighborhoods Now. Read an interview with Ian Gray-Stack, Director of Community Organizing at Banana Kelly. The South Bronx working group will reactivate community gardens in Longwood, Hunts Point, Morrisania, and Mott Haven, allowing for safe outdoor activities and services that address neighborhood needs. Tell us a little bit about your organization. Ian Gray-Stack: Battling city intentions to demolish every building